In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/12 21:04:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


---

In [2]:
# Datafile: BreadBasket_DMS.csv

In [3]:
df = spark.read\
    .option('delimiter',',')\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("shared/hw2/BreadBasket_DMS.csv")

In [4]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Transaction: integer (nullable = true)
 |-- Item: string (nullable = true)



In [5]:
df.head()

Row(Date=datetime.datetime(2016, 10, 30, 0, 0), Time=datetime.datetime(2023, 3, 12, 9, 58, 11), Transaction=1, Item='Bread')

In [6]:
df.show(5)

+-------------------+-------------------+-----------+-------------+
|               Date|               Time|Transaction|         Item|
+-------------------+-------------------+-----------+-------------+
|2016-10-30 00:00:00|2023-03-12 09:58:11|          1|        Bread|
|2016-10-30 00:00:00|2023-03-12 10:05:34|          2| Scandinavian|
|2016-10-30 00:00:00|2023-03-12 10:05:34|          2| Scandinavian|
|2016-10-30 00:00:00|2023-03-12 10:07:57|          3|Hot chocolate|
|2016-10-30 00:00:00|2023-03-12 10:07:57|          3|          Jam|
+-------------------+-------------------+-----------+-------------+
only showing top 5 rows



### Question 1

What is the most popular (most sold) between the 8:00AM and 8:59AM for each day?

In [7]:
from pyspark.sql.functions import hour

df_grouped = df.filter((hour("Time") == 8))\
    .groupBy("Date", "Item")\
    .agg({"Transaction": "count"})\
    .withColumnRenamed("count(Transaction)", "Count")

df_grouped.printSchema()
df_grouped.show(5)

root
 |-- Date: timestamp (nullable = true)
 |-- Item: string (nullable = true)
 |-- Count: long (nullable = false)



+-------------------+-------+-----+
|               Date|   Item|Count|
+-------------------+-------+-----+
|2016-11-05 00:00:00|   NONE|    1|
|2017-03-09 00:00:00| Coffee|    2|
|2017-04-04 00:00:00|Cookies|    1|
|2017-04-06 00:00:00| Coffee|    2|
|2017-01-28 00:00:00|Brownie|    1|
+-------------------+-------+-----+
only showing top 5 rows



In [8]:
from pyspark.sql.functions import max

df_max_count = df_grouped.groupBy("Date")\
    .agg(max("Count").alias("MaxCount"))

df_max_count.printSchema()
df_max_count.show(5)

root
 |-- Date: timestamp (nullable = true)
 |-- MaxCount: long (nullable = true)



+-------------------+--------+
|               Date|MaxCount|
+-------------------+--------+
|2016-11-18 00:00:00|       6|
|2016-11-02 00:00:00|       8|
|2017-03-30 00:00:00|       3|
|2017-04-08 00:00:00|       4|
|2016-12-01 00:00:00|       1|
+-------------------+--------+
only showing top 5 rows



In [9]:
df_result = df_grouped.join(df_max_count, "Date")\
    .filter(df_grouped.Count == df_max_count.MaxCount)\
    .drop('Count', 'MaxCount')\

df_result.printSchema()
df_result.show(5)

root
 |-- Date: timestamp (nullable = true)
 |-- Item: string (nullable = true)

+-------------------+----------------+
|               Date|            Item|
+-------------------+----------------+
|2016-11-18 00:00:00|          Coffee|
|2016-11-02 00:00:00|          Coffee|
|2017-03-30 00:00:00|Christmas common|
|2017-03-30 00:00:00|          Coffee|
|2017-04-08 00:00:00|           Bread|
+-------------------+----------------+
only showing top 5 rows



In [10]:
df_result.sort('Date').show()

+-------------------+---------+
|               Date|     Item|
+-------------------+---------+
|2016-10-31 00:00:00|    Bread|
|2016-10-31 00:00:00|   Coffee|
|2016-11-01 00:00:00|      Tea|
|2016-11-02 00:00:00|   Coffee|
|2016-11-03 00:00:00|   Coffee|
|2016-11-04 00:00:00|    Bread|
|2016-11-04 00:00:00|   Coffee|
|2016-11-05 00:00:00|    Bread|
|2016-11-07 00:00:00|   Coffee|
|2016-11-07 00:00:00|   Pastry|
|2016-11-08 00:00:00|   Coffee|
|2016-11-08 00:00:00|    Bread|
|2016-11-08 00:00:00|   Pastry|
|2016-11-09 00:00:00|   Coffee|
|2016-11-09 00:00:00|   Pastry|
|2016-11-09 00:00:00|    Bread|
|2016-11-10 00:00:00|   Coffee|
|2016-11-11 00:00:00|    Bread|
|2016-11-12 00:00:00|   Coffee|
|2016-11-12 00:00:00|Medialuna|
+-------------------+---------+
only showing top 20 rows



### Question 2

What is the most common item bought along with “Brownie”? (items bought in the same transaction)

In [11]:
from pyspark.sql.functions import count

df_grouped = df.filter(df.Item == "Brownie")\
    .groupBy("Transaction")\
    .count()

df_grouped.printSchema()
df_grouped.show(5)

root
 |-- Transaction: integer (nullable = true)
 |-- count: long (nullable = false)

+-----------+-----+
|Transaction|count|
+-----------+-----+
|       1238|    1|
|       3749|    1|
|       7554|    1|
|       8389|    1|
|        392|    1|
+-----------+-----+
only showing top 5 rows



In [12]:
df_counted = df_grouped.join(df, "Transaction")\
    .groupBy("Item")\
    .agg(count("Transaction").alias("Count"))

df_counted.printSchema()
df_counted.show(5)

root
 |-- Item: string (nullable = true)
 |-- Count: long (nullable = false)

+-----------------+-----+
|             Item|Count|
+-----------------+-----+
|           Muffin|   18|
|            Salad|    1|
|    Bowl Nic Pitt|    1|
|        Alfajores|   27|
|Hearty & Seasonal|    6|
+-----------------+-----+
only showing top 5 rows



In [13]:
from pyspark.sql.functions import desc

df_counted.sort(desc('Count')).show(5)

+-------+-----+
|   Item|Count|
+-------+-----+
|Brownie|  379|
| Coffee|  237|
|  Bread|  115|
|    Tea|   71|
|   Cake|   43|
+-------+-----+
only showing top 5 rows



In [14]:
df_counted.filter(df_counted.Item != "Brownie")\
    .orderBy(df_counted.Count.desc())\
    .first()["Item"]

'Coffee'

---

In [15]:
# Dataset: Restaurants_in_Durham_County_NC.csv

In [16]:
df = spark.read\
    .option('delimiter',';')\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("shared/hw2/Restaurants_in_Durham_County_NC.csv")

In [17]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Premise_Name: string (nullable = true)
 |-- Premise_Address1: string (nullable = true)
 |-- Premise_Address2: string (nullable = true)
 |-- Premise_City: string (nullable = true)
 |-- Premise_State: string (nullable = true)
 |-- Premise_Zip: string (nullable = true)
 |-- Premise_Phone: string (nullable = true)
 |-- Hours_Of_Operation: string (nullable = true)
 |-- Opening_Date: string (nullable = true)
 |-- Closing_Date: string (nullable = true)
 |-- Seats: string (nullable = true)
 |-- Water: string (nullable = true)
 |-- Sewage: string (nullable = true)
 |-- Insp_Freq: string (nullable = true)
 |-- Est_Group_Desc: string (nullable = true)
 |-- Risk: integer (nullable = true)
 |-- Smoking_Allowed: string (nullable = true)
 |-- Type_Description: string (nullable = true)
 |-- Rpt_Area_Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Transitional_Type_Desc: string (nullable = true)
 |-- geolocation: string (nullable = t

In [18]:
df.head()

Row(ID='56060', Premise_Name='WEST 94TH ST PUB', Premise_Address1='4711 HOPE VALLEY RD', Premise_Address2='SUITE 6C', Premise_City='DURHAM', Premise_State='NC', Premise_Zip='27707', Premise_Phone='(919) 403-0025', Hours_Of_Operation=None, Opening_Date='1994-09-01', Closing_Date=None, Seats='60', Water='5 - Municipal/Community', Sewage='3 - Municipal/Community', Insp_Freq='4', Est_Group_Desc='Full-Service Restaurant', Risk=4, Smoking_Allowed='NO', Type_Description='1 - Restaurant', Rpt_Area_Desc='Food Service', Status='ACTIVE', Transitional_Type_Desc='FOOD', geolocation='35.9207272, -78.9573299')

In [19]:
df.show(5)

+-----+--------------------+--------------------+----------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+--------------------+----------------+------+----------------------+--------------------+
|   ID|        Premise_Name|    Premise_Address1|Premise_Address2|Premise_City|Premise_State|Premise_Zip| Premise_Phone|Hours_Of_Operation|Opening_Date|Closing_Date|Seats|               Water|              Sewage|Insp_Freq|      Est_Group_Desc|Risk|Smoking_Allowed|    Type_Description|   Rpt_Area_Desc|Status|Transitional_Type_Desc|         geolocation|
+-----+--------------------+--------------------+----------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+--------------------+----

### Question 3

How many years are represented in this dataset?

In [20]:
from pyspark.sql.functions import to_date, year

df_clean = df.withColumn('Opening_Date', to_date('Opening_Date'))
df_clean = df.withColumn('Closing_Date', to_date('Closing_Date', 'dd-MMM-yyyy'))

df_clean.printSchema()
df_clean.show(5)

root
 |-- ID: string (nullable = true)
 |-- Premise_Name: string (nullable = true)
 |-- Premise_Address1: string (nullable = true)
 |-- Premise_Address2: string (nullable = true)
 |-- Premise_City: string (nullable = true)
 |-- Premise_State: string (nullable = true)
 |-- Premise_Zip: string (nullable = true)
 |-- Premise_Phone: string (nullable = true)
 |-- Hours_Of_Operation: string (nullable = true)
 |-- Opening_Date: string (nullable = true)
 |-- Closing_Date: date (nullable = true)
 |-- Seats: string (nullable = true)
 |-- Water: string (nullable = true)
 |-- Sewage: string (nullable = true)
 |-- Insp_Freq: string (nullable = true)
 |-- Est_Group_Desc: string (nullable = true)
 |-- Risk: integer (nullable = true)
 |-- Smoking_Allowed: string (nullable = true)
 |-- Type_Description: string (nullable = true)
 |-- Rpt_Area_Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Transitional_Type_Desc: string (nullable = true)
 |-- geolocation: string (nullable = tru

In [21]:
years = []

years.extend(df.select(year('Opening_Date').alias('year')).rdd.flatMap(lambda x: x).collect())
years.extend(df.select(year('Closing_Date').alias('year')).rdd.flatMap(lambda x: x).collect())

years = [i for i in years if i is not None]
years = list(set(years))

years

[1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017]

In [22]:
print(years[-1] - years[0])

27


### Question 4

Show the type and count of restaurant opened during the 90’s (1990-1999 inclusive). Note: type=`Rpt_Area_Desc`.

In [23]:
from pyspark.sql.functions import year, count

df.filter(
    (year("Opening_Date") >= 1990) & (year("Opening_Date") <= 1999)
)\
    .groupBy("Rpt_Area_Desc")\
    .count()\
    .show()

+------------------+-----+
|     Rpt_Area_Desc|count|
+------------------+-----+
|Bed&Breakfast Home|    3|
|      Summer Camps|    1|
|      Institutions|   16|
| Local Confinement|    2|
|  School Buildings|   56|
|    Swimming Pools|  256|
|          Day Care|   58|
| Bed&Breakfast Inn|    1|
|           Lodging|   21|
|      Food Service|  204|
+------------------+-----+



---

In [24]:
# Dataset: populationbycountry19802010millions.csv

In [25]:
df = spark.read\
    .option('delimiter',',')\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("shared/hw2/populationbycountry19802010millions.csv")

In [26]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- 1980: string (nullable = true)
 |-- 1981: string (nullable = true)
 |-- 1982: string (nullable = true)
 |-- 1983: string (nullable = true)
 |-- 1984: string (nullable = true)
 |-- 1985: string (nullable = true)
 |-- 1986: string (nullable = true)
 |-- 1987: string (nullable = true)
 |-- 1988: string (nullable = true)
 |-- 1989: string (nullable = true)
 |-- 1990: string (nullable = true)
 |-- 1991: string (nullable = true)
 |-- 1992: string (nullable = true)
 |-- 1993: string (nullable = true)
 |-- 1994: string (nullable = true)
 |-- 1995: string (nullable = true)
 |-- 1996: string (nullable = true)
 |-- 1997: string (nullable = true)
 |-- 1998: string (nullable = true)
 |-- 1999: string (nullable = true)
 |-- 2000: string (nullable = true)
 |-- 2001: string (nullable = true)
 |-- 2002: string (nullable = true)
 |-- 2003: string (nullable = true)
 |-- 2004: string (nullable = true)
 |-- 2005: string (nullable = true)
 |-- 2006: string (nulla

In [27]:
df.head()

23/03/12 21:04:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/03/12 21:04:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv


Row(_c0='North America', 1980='320.27638', 1981='324.44694', 1982='328.62014', 1983='332.72487', 1984='336.72143', 1985='340.74811', 1986='344.89548', 1987='349.07829', 1988='353.2939', 1989='357.68457', 1990='362.4468', 1991='367.70684', 1992='373.29069', 1993='378.74233', 1994='383.9166', 1995='388.97216', 1996='393.9428', 1997='398.97205', 1998='403.85585', 1999='408.60296', 2000='413.3245', 2001='417.83236', 2002='422.05268', 2003='426.06238', 2004='430.26938', 2005='434.47232', 2006='438.82964', 2007='443.3473', 2008='447.67394', 2009='451.83698', 2010='456.59331')

In [28]:
df.show(5)

23/03/12 21:04:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv
+-------------+---------+---------+---------+---------+---------+---------+---------+---------+--------+---------+--------+---------+---------+---------+--------+---------+--------+---------+---------+---------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|          _c0|     1980|     1981|     1982|     1983|     1984|     1985|     1986|     198

### Question 5

For region, compute the percentage change in population, year over year. Note the year 1980 will not have a preceding year. For each year, display the region with the top population decrease.

In [29]:
from pyspark.sql.functions import col, round

result = []

for previous_year, current_year in zip(df.columns[1:], df.columns[2:]):
    year_range = previous_year + "-" + current_year
    df_year_range = df.select("_c0", previous_year, current_year)
    df_year_range = df_year_range.withColumn(
        year_range, 
        round(((col(current_year) - col(previous_year)) / col(previous_year) * 100 ), 2)
    ).na.drop()
    df_year_range.sort(col(year_range)).show(5)
    df_top = df_year_range.orderBy(col(year_range)).first()
    result.append(df_top["_c0"] + "," + str(df_top[year_range]) + "%")

23/03/12 21:04:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981
 Schema: _c0, 1980, 1981
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv
+------------+-------+--------+---------+
|         _c0|   1980|    1981|1980-1981|
+------------+-------+--------+---------+
| Afghanistan|15.0436|13.67368|    -9.11|
|Cook Islands|0.01801| 0.01765|     -2.0|
| El Salvador|4.57041| 4.51972|    -1.11|
|    Dominica|0.07389| 0.07352|     -0.5|
|    Suriname|0.35408| 0.35251|    -0.44|
+------------+-------+--------+---------+
only showing top 5 rows

23/03/12 21:04:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981
 Schema: _c0, 1980, 1981
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv
23/03/12 21:04:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1981, 1982
 Schema: _c0, 1

In [30]:
year = 1980
for top in result:
    print(year, top)
    year = year + 1

1980 Afghanistan,-9.11%
1981 Afghanistan,-8.02%
1982 Antigua and Barbuda,-3.51%
1983 Antigua and Barbuda,-1.75%
1984 Cook Islands,-1.41%
1985 Netherlands Antilles,-24.59%
1986 Saint Helena,-21.3%
1987 Mozambique,-2.88%
1988 Somalia,-2.2%
1989 Liberia,-12.82%
1990 Kuwait,-55.45%
1991 Somalia,-5.39%
1992 Bosnia and Herzegovina,-7.07%
1993 Rwanda,-14.36%
1994 Rwanda,-15.87%
1995 Montserrat,-22.59%
1996 Montserrat,-25.16%
1997 Montserrat,-43.19%
1998 Cook Islands,-2.99%
1999 Cook Islands,-3.26%
2000 Cook Islands,-3.56%
2001 Cook Islands,-3.69%
2002 Montserrat,-6.65%
2003 Djibouti,-4.83%
2004 Montserrat,-8.67%
2005 Nauru,-4.4%
2006 Nauru,-4.7%
2007 Cook Islands,-3.31%
2008 Cook Islands,-3.26%
2009 Cook Islands,-3.2%


---

In [31]:
# Dataset: romeo-juliet-pg1777.txt

In [32]:
txt = spark.read.text("shared/hw2/romeo-juliet-pg1777.txt")

In [33]:
txt.printSchema()

root
 |-- value: string (nullable = true)



In [34]:
txt.head()

Row(value='')

In [35]:
txt.show(5)

+--------------------+
|               value|
+--------------------+
|                    |
|This Etext file i...|
|cooperation with ...|
|Future and Shakes...|
|Etexts that are N...|
+--------------------+
only showing top 5 rows



### Question 6

Do word count in pyspark. Ignore punctuation, and normalize to lower case. Accept only the characters in this set: `[0-9a-zA-Z]`.

In [36]:
import re

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^0-9a-zA-Z]+', ' ', text)
    return text.strip()

lines = txt.rdd.map(lambda r: r[0])

word_counts = lines.flatMap(lambda line: line.split(" ")) \
                  .map(lambda word: (normalize_text(word), 1)) \
                  .filter(lambda pair: len(pair[0]) > 0) \
                  .reduceByKey(lambda x, y: x + y)

df_wordcount = spark.createDataFrame(word_counts, ['Word', 'Count'])

df_wordcount.printSchema()
df_wordcount.show(5)

root
 |-- Word: string (nullable = true)
 |-- Count: long (nullable = true)

+---------+-----+
|     Word|Count|
+---------+-----+
|     this|  258|
|    etext|   29|
|     file|    3|
|       is|  383|
|presented|    1|
+---------+-----+
only showing top 5 rows



---

In [37]:
# Datasets:
#  Restaurants_in_Durham_County_NC.csv
#  durham-nc-foreclosure-2006-2016.json

### Q7

For each restaurant (`Restaurants_in_Durham_County_NC.csv`) with `status`=`ACTIVE` and `rpt_area_desc`=`Food Service`, show the number of foreclosures (`durham-nc-foreclosure-2006-2016`) within a radius of 1 mile of the restaurant’s coordinates.

Note: You can use the Haversine distance. https://pypi.org/project/haversine/

In [38]:
restaurants = spark.read\
    .option("delimiter", ";")\
    .option("inferSchema", True)\
    .option("header", True)\
    .csv("shared/hw2/Restaurants_in_Durham_County_NC.csv")\
    .repartition(10)

restaurants.printSchema()
restaurants.show(5)

root
 |-- ID: string (nullable = true)
 |-- Premise_Name: string (nullable = true)
 |-- Premise_Address1: string (nullable = true)
 |-- Premise_Address2: string (nullable = true)
 |-- Premise_City: string (nullable = true)
 |-- Premise_State: string (nullable = true)
 |-- Premise_Zip: string (nullable = true)
 |-- Premise_Phone: string (nullable = true)
 |-- Hours_Of_Operation: string (nullable = true)
 |-- Opening_Date: string (nullable = true)
 |-- Closing_Date: string (nullable = true)
 |-- Seats: string (nullable = true)
 |-- Water: string (nullable = true)
 |-- Sewage: string (nullable = true)
 |-- Insp_Freq: string (nullable = true)
 |-- Est_Group_Desc: string (nullable = true)
 |-- Risk: integer (nullable = true)
 |-- Smoking_Allowed: string (nullable = true)
 |-- Type_Description: string (nullable = true)
 |-- Rpt_Area_Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Transitional_Type_Desc: string (nullable = true)
 |-- geolocation: string (nullable = t

In [39]:
foreclosures = spark.read\
    .json("shared/hw2/durham-nc-foreclosure-2006-2016.json")\
    .repartition(10)

foreclosures.printSchema()
foreclosures.show(5)

root
 |-- datasetid: string (nullable = true)
 |-- fields: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- geocode: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- parcel_number: string (nullable = true)
 |    |-- year: string (nullable = true)
 |-- geometry: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- record_timestamp: string (nullable = true)
 |-- recordid: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|           datasetid|              fields|            geometry|    record_timestamp|            recordid|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|foreclosure-2006-...|{6117 KELVIN DR, ...|{[-78.956498, 36....|2017-03-06T12:41:...|a9bd7d8b9

In [40]:
from pyspark.sql.functions import split

active = restaurants\
    .filter((col("status") == "ACTIVE") & (col("Rpt_Area_Desc") == "Food Service"))\
    .withColumn("Latitude", split(col("geolocation"), ",").getItem(0))\
    .withColumn("Longitude", split(col("geolocation"), ",").getItem(1))

coordinates = foreclosures.select(
    foreclosures.geometry.coordinates[0].alias("long"),
    foreclosures.geometry.coordinates.getItem(1).alias("lat")
)

result = active.crossJoin(coordinates)

result.printSchema()
result.show(5)

root
 |-- ID: string (nullable = true)
 |-- Premise_Name: string (nullable = true)
 |-- Premise_Address1: string (nullable = true)
 |-- Premise_Address2: string (nullable = true)
 |-- Premise_City: string (nullable = true)
 |-- Premise_State: string (nullable = true)
 |-- Premise_Zip: string (nullable = true)
 |-- Premise_Phone: string (nullable = true)
 |-- Hours_Of_Operation: string (nullable = true)
 |-- Opening_Date: string (nullable = true)
 |-- Closing_Date: string (nullable = true)
 |-- Seats: string (nullable = true)
 |-- Water: string (nullable = true)
 |-- Sewage: string (nullable = true)
 |-- Insp_Freq: string (nullable = true)
 |-- Est_Group_Desc: string (nullable = true)
 |-- Risk: integer (nullable = true)
 |-- Smoking_Allowed: string (nullable = true)
 |-- Type_Description: string (nullable = true)
 |-- Rpt_Area_Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Transitional_Type_Desc: string (nullable = true)
 |-- geolocation: string (nullable = t

In [41]:
from haversine import haversine, Unit

from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

@udf(returnType=FloatType())
def haversine_dist(lat_x, long_x, lat_y, long_y):
    if long_x== None or lat_x == None or long_y == None or lat_y == None:
        return 2000.0
    return haversine((lat_x, long_x), (float(lat_y), float(long_y)), unit=Unit.MILES)

In [42]:
result = result.withColumn(
    "distance", 
    haversine_dist(
        col("lat"),
        col("long"),
        col("Latitude"),
        col("Longitude")
    ).alias("distance")
)

result.printSchema()
result.show(5)

root
 |-- ID: string (nullable = true)
 |-- Premise_Name: string (nullable = true)
 |-- Premise_Address1: string (nullable = true)
 |-- Premise_Address2: string (nullable = true)
 |-- Premise_City: string (nullable = true)
 |-- Premise_State: string (nullable = true)
 |-- Premise_Zip: string (nullable = true)
 |-- Premise_Phone: string (nullable = true)
 |-- Hours_Of_Operation: string (nullable = true)
 |-- Opening_Date: string (nullable = true)
 |-- Closing_Date: string (nullable = true)
 |-- Seats: string (nullable = true)
 |-- Water: string (nullable = true)
 |-- Sewage: string (nullable = true)
 |-- Insp_Freq: string (nullable = true)
 |-- Est_Group_Desc: string (nullable = true)
 |-- Risk: integer (nullable = true)
 |-- Smoking_Allowed: string (nullable = true)
 |-- Type_Description: string (nullable = true)
 |-- Rpt_Area_Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Transitional_Type_Desc: string (nullable = true)
 |-- geolocation: string (nullable = t

+-----+------------+--------------------+----------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+----------------+-------------+------+----------------------+--------------------+----------+------------+-----------+----------+---------+
|   ID|Premise_Name|    Premise_Address1|Premise_Address2|Premise_City|Premise_State|Premise_Zip| Premise_Phone|Hours_Of_Operation|Opening_Date|Closing_Date|Seats|               Water|              Sewage|Insp_Freq|      Est_Group_Desc|Risk|Smoking_Allowed|Type_Description|Rpt_Area_Desc|Status|Transitional_Type_Desc|         geolocation|  Latitude|   Longitude|       long|       lat| distance|
+-----+------------+--------------------+----------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+-

In [43]:
result = result.filter(col("distance") <= 1.0)

result.printSchema()
result.show(5)

root
 |-- ID: string (nullable = true)
 |-- Premise_Name: string (nullable = true)
 |-- Premise_Address1: string (nullable = true)
 |-- Premise_Address2: string (nullable = true)
 |-- Premise_City: string (nullable = true)
 |-- Premise_State: string (nullable = true)
 |-- Premise_Zip: string (nullable = true)
 |-- Premise_Phone: string (nullable = true)
 |-- Hours_Of_Operation: string (nullable = true)
 |-- Opening_Date: string (nullable = true)
 |-- Closing_Date: string (nullable = true)
 |-- Seats: string (nullable = true)
 |-- Water: string (nullable = true)
 |-- Sewage: string (nullable = true)
 |-- Insp_Freq: string (nullable = true)
 |-- Est_Group_Desc: string (nullable = true)
 |-- Risk: integer (nullable = true)
 |-- Smoking_Allowed: string (nullable = true)
 |-- Type_Description: string (nullable = true)
 |-- Rpt_Area_Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Transitional_Type_Desc: string (nullable = true)
 |-- geolocation: string (nullable = t

+-----+------------+--------------------+----------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+----------------+-------------+------+----------------------+--------------------+----------+------------+-----------+----------+----------+
|   ID|Premise_Name|    Premise_Address1|Premise_Address2|Premise_City|Premise_State|Premise_Zip| Premise_Phone|Hours_Of_Operation|Opening_Date|Closing_Date|Seats|               Water|              Sewage|Insp_Freq|      Est_Group_Desc|Risk|Smoking_Allowed|Type_Description|Rpt_Area_Desc|Status|Transitional_Type_Desc|         geolocation|  Latitude|   Longitude|       long|       lat|  distance|
+-----+------------+--------------------+----------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------

In [44]:
result.select('Premise_name', 'distance').show(5)

23/03/12 21:05:15 WARN ExtractPythonUDFFromJoinCondition: The join condition:(haversine_dist(lat#2302, long#2301, Latitude#2250, Longitude#2275)#2466 <= 1.0) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


+--------------------+----------+
|        Premise_name|  distance|
+--------------------+----------+
|DELECTABLE-DELECT...| 0.9322919|
|       THE BAGEL BAR|0.76186764|
|DANISH MEXICAN RE...|0.37278858|
|EL RODEO OF DURHA...| 0.9912659|
|             LITTLER|0.70784277|
+--------------------+----------+
only showing top 5 rows



In [45]:
result = result.groupBy('Premise_name').count()

result.printSchema()
result.show(5)

root
 |-- Premise_name: string (nullable = true)
 |-- count: long (nullable = false)

23/03/12 21:05:18 WARN ExtractPythonUDFFromJoinCondition: The join condition:(haversine_dist(lat#2302, long#2301, Latitude#2250, Longitude#2275)#2466 <= 1.0) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


+--------------------+-----+
|        Premise_name|count|
+--------------------+-----+
|     DPAC  3RD FLOOR|  267|
|W G PEARSON SCHOO...|  199|
|      EL DORADO'S #6|    5|
|            GRILL 46|   17|
|  COMPARE FOODS DELI|   65|
+--------------------+-----+
only showing top 5 rows



In [46]:
result.orderBy(desc("count")).show(5)

23/03/12 21:05:32 WARN ExtractPythonUDFFromJoinCondition: The join condition:(haversine_dist(lat#2302, long#2301, Latitude#2250, Longitude#2275)#2466 <= 1.0) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


+--------------------+-----+
|        Premise_name|count|
+--------------------+-----+
|              SUBWAY|  882|
|TOP'S CHINA RESTA...|  638|
|    TATER BREAD CAFE|  516|
|LOS PRIMOS SUPERM...|  515|
|     THE COTTON ROOM|  486|
+--------------------+-----+
only showing top 5 rows

